# 基于截断的阅读理解模型

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator
from datasets import Dataset
import pandas as pd

In [3]:
df = pd.read_parquet('数据/QA阅读理解模型.parquet')
df.head()

,context,question,answer
0,范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年...,范廷颂是什么时候被任为主教的？,1963年
1,范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年...,1990年，范廷颂担任什么职务？,1990年被擢升为天主教河内总教区宗座署理
2,范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年...,范廷颂是于何时何地出生的？,范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生
3,范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年...,1994年3月，范廷颂担任什么职务？,1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区...
4,范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年...,范廷颂是何时去世的？,范廷颂于2009年2月22日清晨在河内离世


In [ ]:
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)
dataset

## 1、预处理数据

In [ ]:
model_path = "hfl/chinese-macbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
def process_func(examples):
    token = tokenizer(text=examples["question"],
                      text_pair=examples["context"],
                      max_length=512,
                      truncation="only_second",
                      padding="max_length",
                      return_offsets_mapping=True,
                      )
    start_positions, end_positions = [], []

    for idx, offset in enumerate(token["offset_mapping"]):
        context = examples["context"][idx]
        answer = examples["answer"][idx]

        start_token_pos = end_token_pos = 0

        if answer in context:
            start_index = context.find(answer)
            end_index = start_index + len(answer)

            # 文本索引的最后一位小于开始文本 or 文本索引的开始一位大于结束文本
            for token_id, (start, end) in enumerate(offset):
                if 0 < start <= start_index:
                    start_token_pos = token_id
                elif start_index < end <= end_index:
                    end_token_pos = token_id

        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
    token["start_positions"] = start_positions
    token["end_positions"] = end_positions
    return token

In [ ]:
dataloader = dataset.map(process_func, batched=True)
dataloader

## 2、加载模型

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

## 3、配置参数

In [ ]:
args = TrainingArguments(
    output_dir="model",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    logging_steps=100,
    num_train_epochs=3,
    eval_steps=100,
    save_steps=100,
    save_total_limit=1,
)

## 4、创建训练器

In [ ]:
train = Trainer(
    model=model,
    args=args,
    train_dataset=dataloader["train"],
    eval_dataset=dataloader["test"],
    data_collator=DefaultDataCollator(),
)

## 5、训练

In [18]:
train.train()

Step,Training Loss,Validation Loss
100,2.544000,1.455699
200,1.481100,1.276272
300,1.316000,1.185459
400,1.239300,1.131545
500,0.940800,1.164976
600,0.873500,1.181161
700,0.845000,1.175304
800,0.846500,1.162069
900,0.614100,1.246720
1000,0.561900,1.312552


TrainOutput(global_step=1212, training_loss=1.02303412291083, metrics={'train_runtime': 1504.3949, 'train_samples_per_second': 25.776, 'train_steps_per_second': 0.806, 'total_flos': 1.0132565632708608e+16, 'train_loss': 1.02303412291083, 'epoch': 3.0})

## 5、预测

In [19]:
from transformers import pipeline

In [42]:
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

In [68]:
for _ in range(2):
    result = pipe(context=
    [
        "在论文中，作者还分别对浅层和深层的GCN进行了一些实验。在下图中，我们可以看到使用2层或3层的模型可以得到最好的结果。此外，对于深层的GCN（超过7层），反而往往得到不好的性能（虚线蓝色）。"
    ],
        question=["在GCN多少层模型反而效果很差？"])
    print(result)

{'score': 0.0601234957575798, 'start': 62, 'end': 76, 'answer': '对于深层的GCN（超过7层）'}
{'score': 0.061391398310661316, 'start': 71, 'end': 75, 'answer': '超过7层'}


/home/lixiaoying/anaconda3/envs/AiMedJupyter/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
